# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:

- Practice your SQL knowledge

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [1]:
!ls -l

total 260
-rw-r--r-- 1 communal-frame-6742 rvm   1809 Jun 27 18:11 CONTRIBUTING.md
-rw-r--r-- 1 communal-frame-6742 rvm      0 Jun 27 18:11 data
-rw-r--r-- 1 communal-frame-6742 rvm 229376 Jun 27 18:11 data.sqlite
drwxr-xr-x 2 communal-frame-6742 rvm   4096 Jun 27 18:11 images
-rw-r--r-- 1 communal-frame-6742 rvm  18163 Jun 27 18:11 index.ipynb
-rw-r--r-- 1 communal-frame-6742 rvm   1348 Jun 27 18:11 LICENSE.md
-rw-r--r-- 1 communal-frame-6742 rvm   2642 Jun 27 18:11 README.md


In [2]:
import pandas as pd
import sqlite3
con = sqlite3.connect("data.sqlite")
c = con.cursor()

In [3]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('orderdetails',), ('payments',), ('offices',), ('orders',), ('productlines',), ('products',), ('employees',), ('customers',)]


## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [4]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute("""SELECT contactFirstName, contactLastName, phone, addressLine1, addressLine2, creditLimit FROM customers WHERE creditLimit>25000""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,contactFirstName,contactLastName,phone,addressLine1,addressLine2,creditLimit
0,Jean,King,7025551838,8489 Strong St.,,71800.0
1,Peter,Ferguson,03 9520 4555,636 St Kilda Road,Level 3,117300.0
2,Janine,Labrune,40.67.8555,"67, rue des Cinquante Otages",,118200.0
3,Jonas,Bergulfsen,07-98 9555,Erling Skakkes gate 78,,81700.0
4,Susan,Nelson,4155551450,5677 Strong St.,,210500.0


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [5]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute("""SELECT state, AVG(creditLimit) FROM customers GROUP BY state""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [15]:
c.execute("""SELECT customers.customerName, customers.customerNumber, orders.orderNumber, orders.status, orderdetails.quantityOrdered FROM customers INNER JOIN orders ON customers.customerNumber = orders.customerNumber INNER JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(5)

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Online Diecast Creations Co.,363,10100,Shipped,22
1,Online Diecast Creations Co.,363,10100,Shipped,30
2,Online Diecast Creations Co.,363,10100,Shipped,49
3,Online Diecast Creations Co.,363,10100,Shipped,50
4,"Blauer See Auto, Co.",128,10101,Shipped,25


#### Expected Output

<img src='images/expected-output-3.png'>

## HAVING and ORDER BY

Now, return the customerName, customrerNumber, productName, productCode and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [21]:
c.execute("""SELECT customerName, customers.customerNumber, products.productName, products.productName, SUM(orderdetails.quantityOrdered) AS totalOrdered FROM customers INNER JOIN orders ON customers.customerNumber = orders.customerNumber INNER JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber INNER JOIN products on orderdetails.ProductCode = products.productCode GROUP BY products.productCode HAVING SUM(orderdetails.quantityOrdered) >= 10 ORDER BY totalOrdered DESC""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,productName,productName,totalOrdered
0,La Rochelle Gifts,119,1992 Ferrari 360 Spider red,1992 Ferrari 360 Spider red,1808
1,Diecast Classics Inc.,157,1937 Lincoln Berline,1937 Lincoln Berline,1111
2,L'ordine Souveniers,386,American Airlines: MD-11S,American Airlines: MD-11S,1085
3,"Australian Collectables, Ltd",471,1941 Chevrolet Special Deluxe Cabriolet,1941 Chevrolet Special Deluxe Cabriolet,1076
4,L'ordine Souveniers,386,1930 Buick Marquette Phaeton,1930 Buick Marquette Phaeton,1074


#### Expected Output

<img src='images/expected-output-4.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [25]:
#c.execute("""SELECT firstName, lastName, employeeNumber, officeCode FROM employees WHERE officeCode IN (SELECT officeCode FROM employees GROUP BY o)""")
c.execute("""SELECT officeCode, COUNT(employeeNumber) AS numberOfEmployees from employees GROUP BY officeCode HAVING numberOfEmployees < 5""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,officeCode,numberOfEmployees
0,2,2
1,3,2
2,5,2
3,6,4
4,7,2


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!